# Quantum Convolutional Neural Network

This is an implementation of 8 Qubit QCNN circuit testing various ansatze.

In [1]:
import pennylane as qml
from pennylane import numpy as np
np.random.seed(0)

# QCNN Circuit

Here are various unitary ansatze to test

In [2]:
# Unitraies for Convolutional Layers 
def U_TTN(params, wires): # 2 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_5(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRZ(params[4], wires = [wires[1], wires[0]])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])
    
def U_6(params, wires): # 10 params
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])
    qml.RZ(params[2], wires = wires[0])
    qml.RZ(params[3], wires = wires[1])
    qml.CRX(params[4], wires = [wires[1], wires[0]])
    qml.CRX(params[5], wires = [wires[0], wires[1]])
    qml.RX(params[6], wires = wires[0])
    qml.RX(params[7], wires = wires[1])
    qml.RZ(params[8], wires = wires[0])
    qml.RZ(params[9], wires = wires[1])

def U_9(params, wires): # 2 params
    qml.Hadamard(wires = wires[0])
    qml.Hadamard(wires = wires[1])
    qml.CZ(wires = [wires[0],wires[1]])
    qml.RX(params[0], wires = wires[0])
    qml.RX(params[1], wires = wires[1])

def U_13(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRZ(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRZ(params[5], wires = [wires[0], wires[1]])

def U_14(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CRX(params[2], wires = [wires[1], wires[0]])
    qml.RY(params[3], wires = wires[0])
    qml.RY(params[4], wires = wires[1])
    qml.CRX(params[5], wires = [wires[0], wires[1]])

def U_15(params, wires): # 4 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[1], wires[0]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])

def U_SO4(params, wires): # 6 params
    qml.RY(params[0], wires = wires[0])
    qml.RY(params[1], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[2], wires = wires[0])
    qml.RY(params[3], wires = wires[1])
    qml.CNOT(wires = [wires[0], wires[1]])
    qml.RY(params[4], wires = wires[0])
    qml.RY(params[5], wires = wires[1])

# Unitraies for Pooling and Fully Connected Layers
def V_0(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

def V_1(theta, wires):
    qml.CRX(theta, wires = [wires[0], wires[1]])

def F(theta, wires):
    qml.CRZ(theta, wires = [wires[0], wires[1]])

Define general layers that will be used in the circuits.

In [3]:
# Convolution Layer1
def conv_layer1(U, params):
    U(params, wires = [0,7])
    for i in range (0,8,2):
        U(params, wires = [i, i+1])
    for i in range (1,7,2):
        U(params, wires = [i, i+1])
    
def conv_layer2(U, params):
    U(params, wires = [0,2])
    U(params, wires = [4,6])
    U(params, wires = [2,4])
    U(params, wires = [0,6])
    
def pooling_layer1(V_0, V_1, params):
    for i in range(0,8,2):
        V_0(params[0], wires = [i+1, i])
    for i in range(0,8,2):
        qml.PauliX(wires = i+1)
    for i in range(0,8,2):
        V_1(params[1], wires = [i+1, i])
        

def pooling_layer2(V_0, V_1, params): # 2params
    V_0(params[0], wires = [2,0])
    V_0(params[0], wires = [6,4])
    
    qml.PauliX(wires = 2)
    qml.PauliX(wires = 6)
    
    V_1(params[1], wires = [2,0])
    V_1(params[1], wires = [6,4])

Here we define various possible embedding methods.

In [4]:
from pennylane.templates.embeddings import AmplitudeEmbedding, AngleEmbedding

def data_embedding(X, embedding_type = 'Amplitude'):
    if embedding_type == 'Amplitude':
        AmplitudeEmbedding(X, wires = range(8), normalize = True)
    elif embedding_type == 'Angle':
        AngleEmbedding(X, wires = range(8), rotation = 'Y')

Define QCNN circuit with given Unitary Ansatz and embedding method.

In [5]:
dev = qml.device('default.qubit', wires = 8)

@qml.qnode(dev)
def QCNN(X, params, U, U_params, embedding_type = 'Amplitude'):

    param1 = params[0:U_params]
    param2 = params[U_params:U_params + 2]
    param3 = params[U_params + 2: 2*U_params + 2]
    param4 = params[2*U_params + 2: 2*U_params + 4]
    param5 = params[2*U_params + 4]
    
    # Data Embedding
    data_embedding(X, embedding_type = embedding_type)
    
    #Quantum Convolutional Neural Network
    if U == 'U_TTN':
        conv_layer1(U_TTN, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_TTN, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_5':
        conv_layer1(U_5, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_5, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_6':
        conv_layer1(U_6, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_6, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_9':
        conv_layer1(U_9, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_9, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_13':
        conv_layer1(U_13, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_13, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_14':
        conv_layer1(U_14, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_14, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    elif U == 'U_15':
        conv_layer1(U_15, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_15, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
    
    elif U == 'U_SO4':
        conv_layer1(U_SO4, param1)
        pooling_layer1(V_0, V_1, param2)
        conv_layer2(U_SO4, param3)
        pooling_layer2(V_0, V_1, param4)
        F(param5, wires = [0,4])
        
    return qml.expval(qml.PauliZ(4))

# Training Quantum Circuits

In [6]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

def cost(params, X, Y, U, U_params, embedding_type):
    predictions = [QCNN(x, params, U, U_params, embedding_type) for x in X]
    return square_loss(Y, predictions)

def accuracy_test_binary(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.5:
            acc = acc + 1
    return acc / len(labels)

def accuracy_test_one_class(predictions, labels):
    acc = 0
    for l,p in zip(labels, predictions):
        if np.abs(l - p) < 0.25:
            acc = acc + 1
    return acc / len(labels)

In [7]:
def circuit_training(X_train, Y_train, U, U_params, embedding_type):
    total_params = U_params * 2 + 2 * 2 + 1
    params = np.random.randn(total_params)
    steps = 150
    learning_rate = 0.1
    batch_size =25
    opt = qml.NesterovMomentumOptimizer(learning_rate)
    
    for it in range(steps):
        batch_index = np.random.randint(0, len(X_train), (batch_size,))
        X_batch = [X_train[i] for i in batch_index]
        Y_batch = [Y_train[i] for i in batch_index]
        params, cost_new = opt.step_and_cost(lambda v: cost(v, X_batch, Y_batch, U, U_params, embedding_type), params)
        if it % 10 == 0:
            print("iteration: ", it, " cost: ", cost_new)
    return params

### MNIST Data loading and processing

Use PCA and Autoencoder to reduce it into 8 features. We test both one-class classification (labeling 0 and 1) and binary classification (labeling -1 and 1).

In [8]:
import tensorflow as tf
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

def data_load_and_process(classes = [0,1], feature_reduction = 'resize256', label01 = False):
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0 #normalize the data
    
    x_train_filter_01 = np.where((y_train == classes[0]) | (y_train == classes[1]))
    x_test_filter_01 = np.where((y_test == classes[0]) | (y_test == classes[1]))
    
    x_train_01, x_test_01 = x_train[x_train_filter_01], x_test[x_test_filter_01]
    y_train_01, y_test_01 = y_train[x_train_filter_01], y_test[x_test_filter_01]
    
    if label01 == True:
        y_train_01 = [1 if y ==classes[0] else 0 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else 0 for y in y_test_01]
    elif label01 == False:
        y_train_01 = [1 if y ==classes[0] else -1 for y in y_train_01]
        y_test_01 = [1 if y ==classes[0] else -1 for y in y_test_01]
        
    
    if feature_reduction == 'resize256':   
        x_train_01 = tf.image.resize(x_train_01[:], (256, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (256, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01) 
        return x_train_01, x_test_01, y_train_01, y_test_01
    
    elif feature_reduction == 'pca8':
        x_train_01 = tf.image.resize(x_train_01[:], (784, 1)).numpy()
        x_test_01 = tf.image.resize(x_test_01[:], (784, 1)).numpy()
        x_train_01, x_test_01 = tf.squeeze(x_train_01), tf.squeeze(x_test_01)
        
        pca = PCA(8)
        x_train_01 = pca.fit_transform(x_train_01)
        x_test_01 = pca.transform(x_test_01)
        
        #Rescale for angle embedding
        x_train_01, x_test_01 = (x_train_01 + 10) * (np.pi / 20), (x_test_01 + 10) * (np.pi / 20)
        
        return x_train_01, x_test_01, y_train_01, y_test_01

    elif feature_reduction == 'autoencoder8':
        latent_dim = 8 
        class Autoencoder(Model):
            def __init__(self, latent_dim):
                super(Autoencoder, self).__init__()
                self.latent_dim = latent_dim   
                self.encoder = tf.keras.Sequential([
                layers.Flatten(),
                  layers.Dense(latent_dim, activation='relu'),
                ])
                self.decoder = tf.keras.Sequential([
                layers.Dense(784, activation='sigmoid'),
                layers.Reshape((28, 28))
                ])
            def call(self, x):
                encoded = self.encoder(x)
                decoded = self.decoder(encoded)
                return decoded
        
        autoencoder = Autoencoder(latent_dim)
        
        autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())
        autoencoder.fit(x_train_01, x_train_01,
                epochs=10,
                shuffle=True,
                validation_data=(x_test_01, x_test_01))
        
        x_train_01, x_test_01 = autoencoder.encoder(x_train_01).numpy(), autoencoder.encoder(x_test_01).numpy()
        #Rescale for Angle Embedding
        x_train_01, x_test_01 = x_train_01 * (np.pi / 50), x_test_01 * (np.pi / 50)
        
        return x_train_01, x_test_01, y_train_01, y_test_01


# Results with -1, 1 labeling

In [9]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4'] 
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']

In [10]:
x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256 = data_load_and_process(classes = [0,1], feature_reduction = 'resize256', label01 = False)
x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8 = data_load_and_process(classes = [0,1], feature_reduction = 'pca8', label01 = False)
x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8 = data_load_and_process(classes = [0,1], feature_reduction = 'autoencoder8', label01 = False)

Train on 12665 samples, validate on 2115 samples
Epoch 1/10
12665/12665 [==============================] - 1s 79us/sample - loss: 0.0638 - val_loss: 0.0341
Epoch 2/10
12665/12665 [==============================] - 1s 61us/sample - loss: 0.0297 - val_loss: 0.0269
Epoch 3/10
12665/12665 [==============================] - 1s 51us/sample - loss: 0.0256 - val_loss: 0.0237
Epoch 4/10
12665/12665 [==============================] - 1s 52us/sample - loss: 0.0229 - val_loss: 0.0216
Epoch 5/10
12665/12665 [==============================] - 1s 65us/sample - loss: 0.0214 - val_loss: 0.0204
Epoch 6/10
12665/12665 [==============================] - 1s 47us/sample - loss: 0.0203 - val_loss: 0.0194
Epoch 7/10
12665/12665 [==============================] - 1s 56us/sample - loss: 0.0196 - val_loss: 0.0188
Epoch 8/10
12665/12665 [==============================] - 1s 45us/sample - loss: 0.0190 - val_loss: 0.0183
Epoch 9/10
12665/12665 [==============================] - 1s 47us/sample - loss: 0.0186 - val_l


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [11]:
I = len(Unitaries)
J = len(Encodings)
All_predictions = []

for i in range(I):
    for j in range(J):
        U = Unitaries[i]
        U_params = U_num_params[i]
        Encoding = Encodings[j]
        if Encoding == 'resize256':
            Embedding = 'Amplitude'
            X_train, X_test, Y_train, Y_test = x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256
        elif Encoding == 'pca8':
            Embedding = 'Angle'
            X_train, X_test, Y_train, Y_test = x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8
        elif Encoding == 'autoencoder8':
            Embedding = 'Angle'
            X_train, X_test, Y_train, Y_test = x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8
        
        print("\n")
        print("Loss History for " + U + " " + Encoding)
        trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding)

        predictions = [QCNN(x, trained_params, U, U_params, Embedding) for x in X_test]
        All_predictions.append(predictions)
        accuracy = accuracy_test_binary(predictions, Y_test)
        print("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))



Loss History for U_TTN resize256
iteration:  0  cost:  1.0488391126941692
iteration:  10  cost:  0.7855977653667321
iteration:  20  cost:  0.5451320978516572
iteration:  30  cost:  0.39030319225720794
iteration:  40  cost:  0.4217786455644806
iteration:  50  cost:  0.32228144628451666
iteration:  60  cost:  0.3400435398587876
iteration:  70  cost:  0.24963233465923307
iteration:  80  cost:  0.3183577396228646
iteration:  90  cost:  0.296218797805323
iteration:  100  cost:  0.421072450420218
iteration:  110  cost:  0.3414941621602864
iteration:  120  cost:  0.32091430422345907
iteration:  130  cost:  0.3343608007807502
iteration:  140  cost:  0.4237192546804646
Accuracy for U_TTN resize256 :0.9650118203309692


Loss History for U_TTN pca8
iteration:  0  cost:  1.1008786729173738
iteration:  10  cost:  0.9959903998332031
iteration:  20  cost:  0.6025138854504974
iteration:  30  cost:  0.3668631117784137
iteration:  40  cost:  0.265995544544124
iteration:  50  cost:  0.21254755133721986

iteration:  70  cost:  0.8724131872514819
iteration:  80  cost:  0.8083482968855116
iteration:  90  cost:  0.8786088564263207
iteration:  100  cost:  0.7525071720268737
iteration:  110  cost:  0.815810686244922
iteration:  120  cost:  0.7599772304092974
iteration:  130  cost:  0.6589020092016733
iteration:  140  cost:  0.8522339263131549
Accuracy for U_9 autoencoder8 :0.6789598108747045


Loss History for U_13 resize256
iteration:  0  cost:  1.1959642383479372
iteration:  10  cost:  0.956965716065465
iteration:  20  cost:  0.8831022017898418
iteration:  30  cost:  0.8182105785861589
iteration:  40  cost:  0.7054839443539801
iteration:  50  cost:  0.629699640385038
iteration:  60  cost:  0.6461557550978609
iteration:  70  cost:  0.5858303434161953
iteration:  80  cost:  0.5762715867021879
iteration:  90  cost:  0.5342620340163705
iteration:  100  cost:  0.48039794784999174
iteration:  110  cost:  0.4231050263222274
iteration:  120  cost:  0.4564967675220513
iteration:  130  cost:  0.446

iteration:  140  cost:  0.13763482167614227
Accuracy for U_SO4 pca8 :0.9806146572104019


Loss History for U_SO4 autoencoder8
iteration:  0  cost:  0.8341648427576636
iteration:  10  cost:  0.4075218498084925
iteration:  20  cost:  0.3537046299570368
iteration:  30  cost:  0.32008531967602216
iteration:  40  cost:  0.2433875421027269
iteration:  50  cost:  0.25566592967043233
iteration:  60  cost:  0.30961300958344945
iteration:  70  cost:  0.2583602984950201
iteration:  80  cost:  0.36339446202041387
iteration:  90  cost:  0.21706496491558827
iteration:  100  cost:  0.37117141211850885
iteration:  110  cost:  0.3026409650024808
iteration:  120  cost:  0.30255196521945427
iteration:  130  cost:  0.31474683339312415
iteration:  140  cost:  0.24066738345299638
Accuracy for U_SO4 autoencoder8 :0.9702127659574468


# Result with 0, 1 labeling

In [12]:
Unitaries = ['U_TTN', 'U_5', 'U_6', 'U_9', 'U_13', 'U_14', 'U_15', 'U_SO4']
U_num_params = [2, 10, 10, 4, 6, 6, 4, 6]
Encodings = ['resize256', 'pca8', 'autoencoder8']

In [13]:
x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256 = data_load_and_process(classes = [0,1], feature_reduction = 'resize256', label01 = True)
x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8 = data_load_and_process(classes = [0,1], feature_reduction = 'pca8', label01 = True)
x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8 = data_load_and_process(classes = [0,1], feature_reduction = 'autoencoder8', label01 = True)

Train on 12665 samples, validate on 2115 samples
Epoch 1/10
12665/12665 [==============================] - 1s 60us/sample - loss: 0.0654 - val_loss: 0.0354
Epoch 2/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0302 - val_loss: 0.0271
Epoch 3/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0255 - val_loss: 0.0236
Epoch 4/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0228 - val_loss: 0.0218
Epoch 5/10
12665/12665 [==============================] - 0s 38us/sample - loss: 0.0215 - val_loss: 0.0208
Epoch 6/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0206 - val_loss: 0.0200
Epoch 7/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0198 - val_loss: 0.0193
Epoch 8/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0192 - val_loss: 0.0188
Epoch 9/10
12665/12665 [==============================] - 0s 39us/sample - loss: 0.0188 - val_l


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [14]:
I = len(Unitaries)
J = len(Encodings)
All_predictions = []

for i in range(I):
    for j in range(J):
        U = Unitaries[i]
        U_params = U_num_params[i]
        Encoding = Encodings[j]
        if Encoding == 'resize256':
            Embedding = 'Amplitude'
            X_train, X_test, Y_train, Y_test = x_train_resize256, x_test_resize256, y_train_resize256, y_test_resize256
        elif Encoding == 'pca8':
            Embedding = 'Angle'
            X_train, X_test, Y_train, Y_test = x_train_pca8, x_test_pca8, y_train_pca8, y_test_pca8
        elif Encoding == 'autoencoder8':
            Embedding = 'Angle'
            X_train, X_test, Y_train, Y_test = x_train_autoencoder8, x_test_autoencoder8, y_train_autoencoder8, y_test_autoencoder8
        
        print("\n")
        print("Loss History for " + U + " " + Encoding)
        trained_params = circuit_training(X_train, Y_train, U, U_params, Embedding)

        predictions = [QCNN(x, trained_params, U, U_params, Embedding) for x in X_test]
        All_predictions.append(predictions)
        accuracy = accuracy_test_one_class(predictions, Y_test)
        print("Accuracy for " + U + " " + Encoding + " :" + str(accuracy))



Loss History for U_TTN resize256
iteration:  0  cost:  0.5573055416914392
iteration:  10  cost:  0.47592284211973124
iteration:  20  cost:  0.38454021352228074
iteration:  30  cost:  0.3287868432763972
iteration:  40  cost:  0.3906876741451668
iteration:  50  cost:  0.3254591767706407
iteration:  60  cost:  0.26464788906382936
iteration:  70  cost:  0.32348113621946434
iteration:  80  cost:  0.28152457396779923
iteration:  90  cost:  0.24010356441318115
iteration:  100  cost:  0.4946894280513811
iteration:  110  cost:  0.22413084987802673
iteration:  120  cost:  0.23041194727764236
iteration:  130  cost:  0.27142335611264934
iteration:  140  cost:  0.252062916770049
Accuracy for U_TTN resize256 :0.5446808510638298


Loss History for U_TTN pca8
iteration:  0  cost:  0.5293796051823914
iteration:  10  cost:  0.25308626689798763
iteration:  20  cost:  0.29782894307937896
iteration:  30  cost:  0.2622487225273632
iteration:  40  cost:  0.24068475688458615
iteration:  50  cost:  0.1159721

iteration:  50  cost:  0.13258679806367982
iteration:  60  cost:  0.13370129717293908
iteration:  70  cost:  0.11964530209619008
iteration:  80  cost:  0.11057637507376789
iteration:  90  cost:  0.1473923455484133
iteration:  100  cost:  0.1721146550661051
iteration:  110  cost:  0.15239956143675237
iteration:  120  cost:  0.1233205996118979
iteration:  130  cost:  0.13997485576391927
iteration:  140  cost:  0.09389725149270065
Accuracy for U_9 autoencoder8 :0.8132387706855791


Loss History for U_13 resize256
iteration:  0  cost:  0.3562519855466261
iteration:  10  cost:  0.29014633535254436
iteration:  20  cost:  0.27735295596134607
iteration:  30  cost:  0.2695563039890081
iteration:  40  cost:  0.23877905888675122
iteration:  50  cost:  0.22002072513660675
iteration:  60  cost:  0.1832890203176044
iteration:  70  cost:  0.19923897974374433
iteration:  80  cost:  0.2064191681053253
iteration:  90  cost:  0.17114822284191272
iteration:  100  cost:  0.15811204735797604
iteration:  110

iteration:  100  cost:  0.01576929677526146
iteration:  110  cost:  0.03291736999058849
iteration:  120  cost:  0.018594250085627848
iteration:  130  cost:  0.05446988923954546
iteration:  140  cost:  0.08520968191475382
Accuracy for U_SO4 pca8 :0.975886524822695


Loss History for U_SO4 autoencoder8
iteration:  0  cost:  0.49397023524551203
iteration:  10  cost:  0.2078699465814126
iteration:  20  cost:  0.12611689957412606
iteration:  30  cost:  0.11259576251162212
iteration:  40  cost:  0.10874367555873539
iteration:  50  cost:  0.08893490902839962
iteration:  60  cost:  0.08388996064704114
iteration:  70  cost:  0.12226833834765849
iteration:  80  cost:  0.10453468890210862
iteration:  90  cost:  0.17291337336099918
iteration:  100  cost:  0.09350857961720668
iteration:  110  cost:  0.11928564789610716
iteration:  120  cost:  0.09354529766119123
iteration:  130  cost:  0.09598333495364003
iteration:  140  cost:  0.09848732825948757
Accuracy for U_SO4 autoencoder8 :0.943262411347517